# Получаем все видео и делаем список (video_id, video_title, v_year_views, v_pub_datetime)

In [1]:
import polars as pl

df = pl.read_parquet('our_datasets/all_videos.parquet')
data = [(row['video_id'], {'text': row['video_title'], 'v_year_views':row['v_year_views'], 'v_pub_datetime': row['v_pub_datetime']}) for row in df.rows(named=True)]


In [3]:
del df

In [4]:
import gc
gc.collect()

3088

# Создаем Embeddings с контентом и делаем его keyword, индексируем данными (около 45 минут)

In [5]:
import txtai

emb = txtai.Embeddings(content=True, keyword=True)
emb.index(data)
emb.info()

{
  "content": true,
  "keyword": true,
  "offset": 34404396,
  "scoring": {
    "method": "bm25",
    "normalize": true,
    "terms": true
  }
}


In [12]:
del data
gc.collect()

2

# Индекс лучше сохранить

In [13]:
#emb.save('our_indexes/all_videos')

In [2]:
import txtai
emb = txtai.Embeddings()
emb.load('our_indexes/all_videos')

# Поиск с высшим score и свежей датой

In [3]:
query = 'дом 2'
limit = 10
emb.search(f'select id, text, v_year_views, v_pub_datetime, score \
from txtai where similar("{query}") \
order by score desc, v_pub_datetime desc \
limit {limit}')

[{'id': 'video_32603146',
  'text': 'Дом – ДОМ-2',
  'v_year_views': 0.0,
  'v_pub_datetime': '2016-11-03 15:29:17',
  'score': 0.2634873735133661},
 {'id': 'video_23528371',
  'text': 'Дом-2 Гимн Дом-2',
  'v_year_views': 2.0,
  'v_pub_datetime': '2023-08-03 09:39:08',
  'score': 0.2611957032412765},
 {'id': 'video_18536331',
  'text': 'Дом-2: "Дом-2" - болото!',
  'v_year_views': 415.0,
  'v_pub_datetime': '2020-08-12 11:36:00',
  'score': 0.2611957032412765},
 {'id': 'video_4451472',
  'text': 'Дом-2: Японский дом-2',
  'v_year_views': 6.0,
  'v_pub_datetime': '2017-03-16 22:04:22',
  'score': 0.2611957032412765},
 {'id': 'video_6212945',
  'text': 'Дом-2: "Дом-2 в космосе"',
  'v_year_views': 2.0,
  'v_pub_datetime': '2017-04-13 21:05:17',
  'score': 0.25105029841909343},
 {'id': 'video_31750941',
  'text': 'Дом-2: Анонс Дом-2 Live',
  'v_year_views': 13.0,
  'v_pub_datetime': '2016-10-25 07:38:35',
  'score': 0.25105029841909343},
 {'id': 'video_30150801',
  'text': 'Дом-2: Это До

In [47]:
from sklearn.metrics import ndcg_score

In [50]:
test = pl.read_parquet('our_datasets/test_dataset_top_2000.parquet')
test

query,video_id,video_title,count
str,str,str,u32
""" channel296464…","""video_8845225""","""Отдыхает разум…",1873
""" channel296464…","""video_30032971…","""Thai Ridgeback…",6
""" Битва сильней…","""video_4414305""","""Экстрасенсы. Б…",24
""" Битва сильней…","""video_25036288…","""Экстрасенсы. Б…",23
""" Битва сильней…","""video_10854637…","""Экстрасенсы. Б…",23
""" Битва сильней…","""video_18764678…","""Экстрасенсы. Б…",20
""" Битва сильней…","""video_2599212""","""Экстрасенсы. Б…",19
""" Битва сильней…","""video_31312854…","""Экстрасенсы. Б…",17
""" Битва сильней…","""video_25347986…","""Экстрасенсы. Б…",16


In [53]:
sub=[]
for q in test['query'].unique()[:30]:
    sub.append(list(map(lambda x: [q,x['id']],emb.search(q, 5))))
    

In [54]:
sub

[[['Спартак ЦСКА', 'video_29844362'],
  ['Спартак ЦСКА', 'video_7924137'],
  ['Спартак ЦСКА', 'video_8507765'],
  ['Спартак ЦСКА', 'video_4966571'],
  ['Спартак ЦСКА', 'video_13881969']],
 [['маска 4 сезон финал', 'video_26736823'],
  ['маска 4 сезон финал', 'video_14612198'],
  ['маска 4 сезон финал', 'video_33994641'],
  ['маска 4 сезон финал', 'video_17606212'],
  ['маска 4 сезон финал', 'video_21171280']],
 [['адская кухня', 'video_12766367'],
  ['адская кухня', 'video_11940373'],
  ['адская кухня', 'video_14547050'],
  ['адская кухня', 'video_12466567'],
  ['адская кухня', 'video_12464843']],
 [['Мужское женское 2016', 'video_24926278'],
  ['Мужское женское 2016', 'video_30146879'],
  ['Мужское женское 2016', 'video_22973319'],
  ['Мужское женское 2016', 'video_25225500'],
  ['Мужское женское 2016', 'video_3366979']],
 [['уральские пельмени', 'video_12642564'],
  ['уральские пельмени', 'video_20944368'],
  ['уральские пельмени', 'video_24890544'],
  ['уральские пельмени', 'video_1

In [60]:
len(sub[0][0])

2

In [57]:
from sklearn.metrics import ndcg_score
